## Импорт библиотек

In [1]:
import pandas as pd
import requests, json
import ast
import datetime as dt
import time

In [2]:
symbol = 'USDTRUB'
start = '1651363200000' # GMT 1 мая 2022 года 00:00:00
end = dt.datetime.now() # по настоящее время
end = (end - dt.datetime(1970, 1, 1)) // dt.timedelta(milliseconds=1)
limit = '500'
interval = '1h' 

In [3]:
column_list = ['Kline open time','Open price','High price','Low price','Close price',
                            'Volume','Kline Close time','Quote asset volume','Number of trades',
                            'Taker buy base asset volume','Taker buy quote asset volume','Unused field, ignore']

In [4]:
# создадим интерполяционную сетку, чтоб проще было отсчитывать временные отрезки
ti = pd.date_range(start = pd.to_datetime(start, unit='ms'), end = pd.to_datetime(end, unit='ms'), freq="1h") 
ts = pd.DataFrame({'date':list(ti)})
ts['unixtime'] = (ts.date - pd.Timestamp("1970-01-01 00:00:00")) // pd.Timedelta('1ms')

In [5]:
df = pd.DataFrame(columns=column_list)
start_point = 0
end_point = 0
iteration = 0
attempt = 0
data_amount = len(ts)

while end_point < data_amount-1:
    end_point = start_point+500
    if end_point > data_amount:
        end_point = data_amount-1
    startTime = ts.loc[start_point, 'unixtime']
    endTime = ts.loc[end_point, 'unixtime']
    while attempt < 4:
        try:
            url = f'https://api.binance.com/api/v3/klines?startTime={startTime}&endTime={endTime}&limit={limit}&symbol={symbol}&interval={interval}'
            response = requests.get(url)
            data_list = response.content.decode("utf-8")[2:-2]
            data = data_list.split('],[')
            data = [list(ast.literal_eval(x)) for x in data]
            break
        except:
            attempt += 1
            print(f':start_point {start_point}, end_point: {end_point}.')
            print(f'Время ожидания ответа превышено. {attempt}-я попытка')
            time.sleep(1)
    current_df = pd.DataFrame(data, columns=column_list)
    # Запись текущего датасета в файл
    current_df.to_csv(f"df_{iteration}.csv", index=False)
    df = df.append(current_df, ignore_index=True,
                   verify_integrity=False, sort=False)
    iteration += 1
    start_point = end_point

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8815 entries, 0 to 8814
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Kline open time               8815 non-null   object
 1   Open price                    8815 non-null   object
 2   High price                    8815 non-null   object
 3   Low price                     8815 non-null   object
 4   Close price                   8815 non-null   object
 5   Volume                        8815 non-null   object
 6   Kline Close time              8815 non-null   object
 7   Quote asset volume            8815 non-null   object
 8   Number of trades              8815 non-null   object
 9   Taker buy base asset volume   8815 non-null   object
 10  Taker buy quote asset volume  8815 non-null   object
 11  Unused field, ignore          8815 non-null   object
dtypes: object(12)
memory usage: 826.5+ KB


In [7]:
df.insert(1, 'time', pd.to_datetime(df['Kline open time'], unit='ms')) 
df.drop(['Kline Close time', 'Unused field, ignore'], axis = 1, inplace=True)

In [8]:
df

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1651363200000,2022-05-01 00:00:00,72.57000000,72.70000000,72.19000000,72.22000000,44042.00000000,3192083.31000000,288,16793.00000000,1217389.99000000
1,1651366800000,2022-05-01 01:00:00,72.22000000,72.33000000,72.00000000,72.02000000,31815.00000000,2294299.05000000,394,14240.00000000,1027333.02000000
2,1651370400000,2022-05-01 02:00:00,72.04000000,72.14000000,72.00000000,72.09000000,41992.00000000,3024975.76000000,279,17690.00000000,1274526.63000000
3,1651374000000,2022-05-01 03:00:00,72.06000000,72.30000000,71.83000000,72.10000000,41300.00000000,2974221.38000000,202,17820.00000000,1283556.99000000
4,1651377600000,2022-05-01 04:00:00,72.09000000,72.09000000,71.30000000,71.40000000,97018.00000000,6947588.91000000,262,30646.00000000,2200024.93000000
...,...,...,...,...,...,...,...,...,...,...,...
8810,1683079200000,2023-05-03 02:00:00,80.94000000,81.06000000,80.94000000,81.04000000,17458.00000000,1414530.39000000,159,10732.00000000,869598.37000000
8811,1683082800000,2023-05-03 03:00:00,81.04000000,81.05000000,80.96000000,80.96000000,13088.00000000,1059981.35000000,141,6221.00000000,503841.35000000
8812,1683086400000,2023-05-03 04:00:00,80.97000000,80.98000000,80.92000000,80.97000000,25958.00000000,2101452.82000000,170,17376.00000000,1406770.43000000
8813,1683090000000,2023-05-03 05:00:00,80.96000000,81.01000000,80.81000000,80.88000000,31952.00000000,2585147.13000000,256,14698.00000000,1188977.98000000


In [9]:
df.to_csv('df_for_EDA.csv', index=False)